In [1]:
import json
from scantools.capture import Capture

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
capture = Capture.load('data/')

[2024/11/13 17:38:18 scantools INFO] Loading Capture from /lamar/lamar/data.


In [9]:
trajectories = []

point_clouds = ""

for name, session in capture.sessions.items():
    pose = session.proc.alignment_global.get_abs_pose('pose_graph_optimized')
    trajectory = [
        {
            "timestamp": t,
            "sensor_records": [
                {
                    "sensor": name,
                    "position": list(pose.r.apply(sensor.t) + pose.t),
                    "rotation": list((pose.r * sensor.r).as_quat())
                } for name, sensor in record.items()
            ]
        } for t, record in session.trajectories.items()
    ]
        
    trajectories.append({
        "name": name,
        "trajectory": trajectory
    })
    
    if session.pointclouds:
        pose_str = " ".join([str(x) for x in pose.t])
        rot_str = " ".join([str(x) for x in pose.qvec])
        point_clouds += f"data/{name}/raw_data/pointcloud.ply {pose_str} {rot_str}\n"
    

with open("trajectories.json", "w") as f:
    json.dump({"trajectories": trajectories}, f)
    
with open("point_clouds.txt", "w") as f:
    f.write(point_clouds)
    